## Solve a heat equation for variable diffusion D
This script includes D as an input to the network, such that the trained network solves the heat equation for a whole range of diffusion coefficients

In [1]:
import torch
import numpy as np
import pytorch_lightning as pl
from timeit import default_timer as timer

from neural_diff_eq.problem import (Variable,
                                    Setting)
from neural_diff_eq.problem.domain import (Rectangle,
                                           Interval)
from neural_diff_eq.problem.condition import (DirichletCondition,
                                              DiffEqCondition,
                                              DataCondition)
from neural_diff_eq.models.fcn import BlockFCN
from neural_diff_eq import PINNModule
from neural_diff_eq.utils import laplacian, gradient
from neural_diff_eq.utils.fdm import FDM, create_validation_data
from neural_diff_eq.utils.plot import Plotter

from neural_diff_eq.datamodule import ProblemDataModule

os.environ["CUDA_VISIBLE_DEVICES"] = "7"

#pl.seed_everything(43)

In [2]:
w, h = 50, 50
t0, tend = 0, 1
temp_hot = 10
D_low, D_up = 5, 25  # set here the interval boundary for D

We define the independent variables of the PDE, which will later determine the inputs to the NN. Every variable has a domain and can have one or more boundary conditions.

In [3]:
x = Variable(name='x',
             order=2,
             domain=Rectangle(corner_dl=[0, 0],
                              corner_dr=[w, 0],
                              corner_tl=[0, h]),
             train_conditions={},
             val_conditions={})
t = Variable(name='t',
             order=1,
             domain=Interval(low_bound=0,
                             up_bound=tend),
             train_conditions={},
             val_conditions={})
D = Variable(name='D',
             order=0,
             domain=Interval(low_bound=D_low,
                             up_bound=D_up),
             train_conditions={},
             val_conditions={})

Now we add the wanted boundary conditions to x and t and define the norm that will be used in all training conditions.

In [4]:
norm = torch.nn.MSELoss()

def x_dirichlet_fun(input):
    return np.zeros_like(input['t'])

x.add_train_condition(DirichletCondition(dirichlet_fun=x_dirichlet_fun,
                                         name='dirichlet',
                                         norm=norm,
                                         batch_size=500,
                                         dataset_size=500,
                                         num_workers=4,
                                         data_plot_variables=('x','t')))

def t_dirichlet_fun(input):
    return temp_hot*np.sin(np.pi/w*input['x'][:, :1])*np.sin(np.pi/h*input['x'][:, 1:])

t.add_train_condition(DirichletCondition(dirichlet_fun=t_dirichlet_fun,
                                         name='dirichlet',
                                         norm=norm,
                                         batch_size=500,
                                         dataset_size=500,
                                         num_workers=4,
                                         boundary_sampling_strategy='lower_bound_only',
                                         data_plot_variables=('x','t')))


Using the same notation, we can also define a PDE-condition to the inner of the domain.

In [5]:
def pde(u, input):
    return gradient(u, input['t']) - input['D']*laplacian(u, input['x'])

train_cond = DiffEqCondition(pde=pde,
                             norm=norm,
                             batch_size=5000,
                             dataset_size=5000,
                             num_workers=8,
                             data_plot_variables=('x','t'))

For comparison during validation, we solve the problem for some D using a simple FDM scheme:

In [7]:
domain_dic = {'x': [[0, w], [0, h]]}
dx, dy = 0.5, 0.5
step_width_dict = {'x': [dx, dy]}
time_interval = [t0, tend]

D_list = [5, 10, 15, 20, 25]
# ^Here you can add many different values for D, e.g [18.8,2.5,20,....]
# The FDM-Methode will compute solutions for all D.
# For too many D this will become really memory expensive, since
# the FDM uses a forward euler!
fdm_start = timer()
domain, time, u = FDM(domain_dic, step_width_dict, time_interval,
                      D_list, t_dirichlet_fun)
fdm_end = timer()
print('Time for FDM-Solution:', fdm_end-fdm_start)
data_x, data_u = create_validation_data(domain, time, u, D_list, D_is_input=True)
# True: if D is input of the model

class InfNorm(torch.nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, input_a, input_b):
        return torch.max(torch.abs(input_a-input_b))

max_norm = InfNorm()

val_cond = DataCondition(data_x=data_x,
                         data_u=data_u,
                         name='validation',
                         norm=norm,
                         batch_size=len(data_u[:, 0])//100,
                         num_workers=16)

Time for FDM-Solution: 0.46186890499666333


The variables as well as the conditions for the inner part of the domain are collected in a Setting:

In [8]:
setup = Setting(variables=(x, t, D),
                train_conditions={'pde': train_cond},
                val_conditions={'validation': val_cond})

To solve the problem defined by setting using a PINN, we define a solver. A plotter can help us to visualize intermediate results in the tensorboard.

In [9]:
plotter = Plotter(plot_variables=setup.variables['x'],
                  points=400,
                  dic_for_other_variables={'t': 1.0, 'D': 15.0},
                  all_variables=setup.variables,
                  log_interval=1)

solver = PINNModule(model=BlockFCN(input_dim=4,
                                   blocks=2,
                                   width=20),  # TODO: comput input_dim in setting
                    problem=setup,
                    optimizer=torch.optim.Adam,
                    lr=1e-3,
                    #log_plotter=plotter
                    )
datamod = ProblemDataModule(problem=setup,n_iterations=100)

Finally, we define a lightning trainer and train the model:

In [12]:
trainer = pl.Trainer(gpus='-1',
                     #accelerator='ddp',
                     #plugins=pl.plugins.DDPPlugin(find_unused_parameters=False),
                     num_sanity_val_steps=0,
                     check_val_every_n_epoch=100,
                     log_every_n_steps=1,
                     max_epochs=10,
                     # limit_val_batches=10,  # The validation dataset is probably pretty big,
                     # so you need to see how much you want to
                     # check every validation
                     # checkpoint_callback=False)
                     )

trainer.fit(solver, datamod)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [7]

  | Name  | Type     | Params
-----------------------------------
0 | model | BlockFCN | 1.8 K 
-----------------------------------
1.8 K     Trainable params
0         Non-trainable params
1.8 K     Total params
0.007     Total estimated model params size (MB)
Epoch 9: 100%|██████████| 100/100 [00:01<00:00, 51.56it/s, loss=0.132, v_num=122]
